In [1]:
#get data and format
from scipy import stats
import numpy as np
import pandas as pd
from sklearn import *
from math import sqrt
import numpy as np

data = pd.read_csv(r'C:\Users\Pujanande\Desktop\Capstone 1\all_data_cap1.csv') #import data
data['TRADE_DATE'] = pd.to_datetime(data['TRADE_DATE']) #convert to DT
data['FERC_time'] = 0 #'off_peak' #make everthing off peak and then change it to peak if criteria are met
data['FERC_time'][(data['TRADE_DATE'].dt.weekday <= 5) & (data['TRADE_DATE'].dt.hour >= 6) & (data['TRADE_DATE'].dt.hour <= 21)]=1 #'peak' 

data['FERC_season'] = 0 #'shoulder' #make everthing shoulder and then change it to winter or summer if criteria are met
data['FERC_season'][(data['TRADE_DATE'].dt.month <= 2) | (data['TRADE_DATE'].dt.month >= 12)]= 1 # winter' 
data['FERC_season'][(data['TRADE_DATE'].dt.month <= 8) & (data['TRADE_DATE'].dt.month >= 6)]= 2

C:\Users\Pujanande\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Pujanande\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\Pujanande\Anaconda3\lib\site-packages\sklearn\lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
C:\Users\

In [8]:
#create simple prediction function, this time predicting a price for each hour
def simple_price_2(season, time):
    season_name = data.loc[data['FERC_season'] == season] #subset data
    period = season_name.loc[season_name['FERC_time'] == time]

    year_list = data['TRADE_DATE'].dt.year #list unique years in data
    year_list = np.unique(year_list)
    
    df = pd.DataFrame(columns=('mean','std','con_int')) #empty df
    df2 = pd.DataFrame() #empty df
    
    for y in year_list : #for every year
        year_df = period[period['TRADE_DATE'].dt.year == y] #split years
        mean = year_df.PRICEINDOLPERMWH.mean() #mean of prices
        std = year_df.PRICEINDOLPERMWH.std() #std of prices
        con = stats.norm.interval(0.95, loc=mean, scale=std) #95% confidence range
        df.loc[y] = [mean,std,con] #add to df
        df2[y] = year_df.PRICEINDOLPERMWH
    
    df2['pred'] = df2.mean(numeric_only=True, axis=1) #predicted price average of historic
    pred_mean = df2.pred.mean() # mean of predicted
    pred_std = df2.pred.std() # std of predicted
    pred_con = stats.norm.interval(0.95, loc=pred_mean, scale=pred_std) #95% confidence range
        
    df.loc['pred'] = [pred_mean,pred_std,pred_con] #add to df 
        
    return print(df)

In [9]:
simple_price_2(0,0)

           mean       std                         con_int
2014  25.735525  5.117542  (15.7053270736, 35.7657227646)
2015  17.450875  7.154513  (3.42828760952, 31.4734627899)
2016  16.172487  5.932291  (4.54541091099, 27.7995625716)
pred  25.735525  5.117542  (15.7053270736, 35.7657227646)
